# Creating an IMDB sentiment analysis Model

This notebook will create a model and then we will use TCAVs to analyse the model and see what labels make it decide if it's good or bad. 

In [1]:
import tensorflow as tf
import numpy as np
import tensorflow_datasets as tfds
from tensorflow import keras

now let's create the dataset for imdb

In [2]:
dir_path = "/code/tcav/tcav_examples/IMDB_Data/"
imdb,info = tfds.load("imdb_reviews",with_info=True,as_supervised=True,data_dir=dir_path,download=True)

   Get the data ready for training and testing

In [13]:
train_data, test_data = imdb['train'], imdb['test']
training_sentences = []
training_labels = []
TextLabels = []

testing_sentences = []
testing_labels = []
for s,l in train_data:
  TextLabels.append(l)
  training_sentences.append(str(s.numpy()))
  training_labels.append(l.numpy())
  
for s,l in test_data:
  testing_sentences.append(str(s.numpy()))
  testing_labels.append(l.numpy())
  
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

Tokenize the text

In [4]:
vocab_size = 10000
embedding_dim = 16
max_length = 100
trunc_type='post'
oov_tok = "<OOV>"

from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

Doing NLP sequencing of the IMDB data

In [5]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
sequences = tokenizer.texts_to_sequences(training_sentences)

Padding the lenghts so that the input is always the same size

In [6]:
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)

We will now create the Model that will be modified for TCAV later

In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dense (Dense)                (None, 24)                38424     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 25        
Total params: 198,449
Trainable params: 198,449
Non-trainable params: 0
_________________________________________________________________


Now let's train this model

In [8]:
num_epochs = 10
history = model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 3s 3ms/step - loss: 0.5988 - accuracy: 0.6318 - val_loss: 0.4032 - val_accuracy: 0.8154
Epoch 2/10
782/782 [==============================] - 2s 3ms/step - loss: 0.2367 - accuracy: 0.9124 - val_loss: 0.4316 - val_accuracy: 0.8072
Epoch 3/10
782/782 [==============================] - 2s 3ms/step - loss: 0.0712 - accuracy: 0.9842 - val_loss: 0.5805 - val_accuracy: 0.7993
Epoch 4/10
782/782 [==============================] - 2s 3ms/step - loss: 0.0100 - accuracy: 0.9990 - val_loss: 0.6953 - val_accuracy: 0.7971
Epoch 5/10
782/782 [==============================] - 2s 3ms/step - loss: 0.0023 - accuracy: 0.9999 - val_loss: 0.7484 - val_accuracy: 0.8024
Epoch 6/10
782/782 [==============================] - 2s 3ms/step - loss: 7.4804e-04 - accuracy: 1.0000 - val_loss: 0.7991 - val_accuracy: 0.8033
Epoch 7/10
782/782 [==============================] - 2s 3ms/step - loss: 3.7590e-04 - accuracy: 1.0000 - val_loss: 0.8479 - val_accuracy: 0.803

Now let's test out the model!

In [9]:
new_sentences = [
    'I loved this movie.',
    'This film is so boring.',
    'This movie is so hilarious. I had a really great time!',
    'Very linear scenario, no surprises at all',
    'Another amazing addition to the franchise with good story arcs and standalone episodes.',
    'Not for the hardened not even the casual fans.'
    ]
new_sequences = tokenizer.texts_to_sequences(new_sentences)
padded=pad_sequences(new_sequences, maxlen=max_length,truncating=trunc_type)
output=model.predict(padded)
for i in range(0,len(new_sentences)):
    print('Review:'+new_sentences[i]+' '+'sentiment:'+str(output[i])+'\n')

Review:I loved this movie. sentiment:[0.28623527]

Review:This film is so boring. sentiment:[0.0012663]

Review:This movie is so hilarious. I had a really great time! sentiment:[0.9053588]

Review:Very linear scenario, no surprises at all sentiment:[0.1437445]

Review:Another amazing addition to the franchise with good story arcs and standalone episodes. sentiment:[0.9714278]

Review:Not for the hardened not even the casual fans. sentiment:[0.20399252]



In [10]:
model.save(dir_path+"imdb_model.h5")

In [14]:
#labels = open(os.path.join(dir_path,"labels.txt"))
print(TextLabels)

[<tf.Tensor: shape=(), dtype=int64, numpy=0>, <tf.Tensor: shape=(), dtype=int64, numpy=0>, <tf.Tensor: shape=(), dtype=int64, numpy=0>, <tf.Tensor: shape=(), dtype=int64, numpy=1>, <tf.Tensor: shape=(), dtype=int64, numpy=1>, <tf.Tensor: shape=(), dtype=int64, numpy=1>, <tf.Tensor: shape=(), dtype=int64, numpy=0>, <tf.Tensor: shape=(), dtype=int64, numpy=0>, <tf.Tensor: shape=(), dtype=int64, numpy=0>, <tf.Tensor: shape=(), dtype=int64, numpy=0>, <tf.Tensor: shape=(), dtype=int64, numpy=0>, <tf.Tensor: shape=(), dtype=int64, numpy=1>, <tf.Tensor: shape=(), dtype=int64, numpy=1>, <tf.Tensor: shape=(), dtype=int64, numpy=0>, <tf.Tensor: shape=(), dtype=int64, numpy=1>, <tf.Tensor: shape=(), dtype=int64, numpy=0>, <tf.Tensor: shape=(), dtype=int64, numpy=1>, <tf.Tensor: shape=(), dtype=int64, numpy=1>, <tf.Tensor: shape=(), dtype=int64, numpy=1>, <tf.Tensor: shape=(), dtype=int64, numpy=0>, <tf.Tensor: shape=(), dtype=int64, numpy=1>, <tf.Tensor: shape=(), dtype=int64, numpy=1>, <tf.Tenso